****HACKATHON AI PLANET****

**Nutrition Health Survey- Age Prediction Summer Analytics 2025, IIT Guwahati**

In [2]:
import numpy as np

In [3]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('Train_Data.csv')
test_df = pd.read_csv('Test_Data.csv')
sample_submission_df = pd.read_csv('Sample_Submission.csv')

# Display the first few rows of the training data
print("Train Data - Head:")
print(train_df.head())

# Display the information about the training data
print("\nTrain Data - Info:")
print(train_df.info())

# Display the first few rows of the test data
print("\nTest Data - Head:")
print(test_df.head())

# Display the information about the test data
print("\nTest Data - Info:")
print(test_df.info())

# Display the first few rows of the sample submission file
print("\nSample Submission Data - Head:")
print(sample_submission_df.head())

# Display the information about the sample submission file
print("\nSample Submission Data - Info:")
print(sample_submission_df.info())


Train Data - Head:
      SEQN  RIAGENDR  PAQ605  BMXBMI  LBXGLU  DIQ010  LBXGLT  LBXIN age_group
0  73564.0       2.0     2.0    35.7   110.0     2.0   150.0  14.91     Adult
1  73568.0       2.0     2.0    20.3    89.0     2.0    80.0   3.85     Adult
2  73576.0       1.0     2.0    23.2    89.0     2.0    68.0   6.14     Adult
3  73577.0       1.0     2.0    28.9   104.0     NaN    84.0  16.15     Adult
4  73580.0       2.0     1.0    35.9   103.0     2.0    81.0  10.92     Adult

Train Data - Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1966 entries, 0 to 1965
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SEQN       1954 non-null   float64
 1   RIAGENDR   1948 non-null   float64
 2   PAQ605     1953 non-null   float64
 3   BMXBMI     1948 non-null   float64
 4   LBXGLU     1953 non-null   float64
 5   DIQ010     1948 non-null   float64
 6   LBXGLT     1955 non-null   float64
 7   LBXIN      1957 non-nu

In [4]:
from sklearn.impute import SimpleImputer

# Separate features and target variable in training data
X_train = train_df.drop('age_group', axis=1)
y_train = train_df['age_group']

# Copy test data for preprocessing
X_test = test_df.copy()

# Impute missing values using the median strategy
# Using median as it is robust to outliers
imputer = SimpleImputer(strategy='median')

# Fit imputer on training data and transform both training and test data
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Verify that there are no more missing values
print("\nMissing values in X_train_imputed after imputation:")
print(X_train_imputed.isnull().sum())

print("\nMissing values in X_test_imputed after imputation:")
print(X_test_imputed.isnull().sum())

# Encode the target variable: Adult -> 0, Senior -> 1
y_train_encoded = y_train.map({'Adult': 0, 'Senior': 1})

# Verify the encoding
print("\nEncoded age_group counts:")
print(y_train_encoded.value_counts())


Missing values in X_train_imputed after imputation:
SEQN        0
RIAGENDR    0
PAQ605      0
BMXBMI      0
LBXGLU      0
DIQ010      0
LBXGLT      0
LBXIN       0
dtype: int64

Missing values in X_test_imputed after imputation:
SEQN        0
RIAGENDR    0
PAQ605      0
BMXBMI      0
LBXGLU      0
DIQ010      0
LBXGLT      0
LBXIN       0
dtype: int64

Encoded age_group counts:
age_group
0.0    1638
1.0     314
Name: count, dtype: int64


In [5]:
# Drop 'SEQN' column as it's an identifier
X_train_processed = X_train_imputed.drop('SEQN', axis=1)
X_test_processed = X_test_imputed.drop('SEQN', axis=1)

# Identify categorical columns for one-hot encoding
categorical_cols = ['RIAGENDR', 'PAQ605', 'DIQ010']

# Convert specified columns to object type to treat them as categorical
for col in categorical_cols:
    X_train_processed[col] = X_train_processed[col].astype('object')
    X_test_processed[col] = X_test_processed[col].astype('object')

# Apply one-hot encoding
X_train_encoded = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

# Ensure consistent columns between training and test sets after one-hot encoding
# This handles cases where a category might be present in one set but not the other
train_cols = set(X_train_encoded.columns)
test_cols = set(X_test_encoded.columns)

missing_in_test = list(train_cols - test_cols)
for col in missing_in_test:
    X_test_encoded[col] = 0

missing_in_train = list(test_cols - train_cols)
for col in missing_in_train:
    X_train_encoded[col] = 0

# Align columns to ensure the order is the same
X_test_encoded = X_test_encoded[X_train_encoded.columns]

print("\nX_train_encoded - Head:")
print(X_train_encoded.head())

print("\nX_test_encoded - Head:")
print(X_test_encoded.head())

print("\nX_train_encoded - Info:")
print(X_train_encoded.info())

print("\nX_test_encoded - Info:")
print(X_test_encoded.info())


X_train_encoded - Head:
   BMXBMI  LBXGLU  LBXGLT  LBXIN  RIAGENDR_2.0  PAQ605_2.0  PAQ605_7.0  \
0    35.7   110.0   150.0  14.91          True        True       False   
1    20.3    89.0    80.0   3.85          True        True       False   
2    23.2    89.0    68.0   6.14         False        True       False   
3    28.9   104.0    84.0  16.15         False        True       False   
4    35.9   103.0    81.0  10.92          True       False       False   

   DIQ010_2.0  DIQ010_3.0  
0        True       False  
1        True       False  
2        True       False  
3        True       False  
4        True       False  

X_test_encoded - Head:
   BMXBMI  LBXGLU  LBXGLT  LBXIN  RIAGENDR_2.0  PAQ605_2.0  PAQ605_7.0  \
0    32.2    96.0   135.0  15.11         False       False           0   
1    26.3   100.0   141.0  15.26          True        True           0   
2    28.6   107.0   136.0   8.82         False        True           0   
3    22.1    93.0   111.0  12.13          

In [7]:
# Reload original data to ensure correct handling of NaNs in target
train_df = pd.read_csv('Train_Data.csv')

# Drop rows with NaN in 'age_group' from the training data
train_df_cleaned = train_df.dropna(subset=['age_group'])

# Separate features and target variable from the cleaned training data
X_train = train_df_cleaned.drop('age_group', axis=1)
y_train = train_df_cleaned['age_group']

# Copy test data for preprocessing (test_df remains the same as it doesn't have 'age_group')
test_df = pd.read_csv('Test_Data.csv')
X_test = test_df.copy()

# Impute missing values using the median strategy
imputer = SimpleImputer(strategy='median')

# Fit imputer on training data and transform both training and test data
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Drop 'SEQN' column as it's an identifier
X_train_processed = X_train_imputed.drop('SEQN', axis=1)
X_test_processed = X_test_imputed.drop('SEQN', axis=1)

# Identify categorical columns for one-hot encoding
categorical_cols = ['RIAGENDR', 'PAQ605', 'DIQ010']

# Convert specified columns to object type to treat them as categorical
for col in categorical_cols:
    X_train_processed[col] = X_train_processed[col].astype('object')
    X_test_processed[col] = X_test_processed[col].astype('object')

# Apply one-hot encoding
X_train_encoded = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

# Ensure consistent columns between training and test sets after one-hot encoding
train_cols = set(X_train_encoded.columns)
test_cols = set(X_test_encoded.columns)

missing_in_test = list(train_cols - test_cols)
for col in missing_in_test:
    X_test_encoded[col] = 0

missing_in_train = list(test_cols - train_cols)
for col in missing_in_train:
    X_train_encoded[col] = 0

# Align columns to ensure the order is the same
X_test_encoded = X_test_encoded[X_train_encoded.columns]

# Encode the target variable: Adult -> 0, Senior -> 1
y_train_encoded = y_train.map({'Adult': 0, 'Senior': 1})

# Initialize the RandomForestClassifier model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train_encoded, y_train_encoded)

# Make predictions on the test set
predictions = model.predict(X_test_encoded)

# Create the submission DataFrame
submission_df = pd.DataFrame({'age_group': predictions})

# Display the first few rows of the submission file
print("\nSubmission DataFrame - Head:")
print(submission_df.head())

# Save the submission file to CSV
submission_df.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully.")

NameError: name 'RandomForestClassifier' is not defined